# Imports

## Libraries

In [175]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

## Data

In [176]:
from os import listdir

def Locate_Files(path_to_dir, suffix=(".csv")):
    """
    path_to_dir = path to files
    suffix = file extension (must be tuple if there's more than one file extension needed)
    """
    
    #Fetch all Files
    filenames = listdir(path_to_dir)
    
    #Filter all Files
    filenames = [filename for filename in filenames if filename.endswith(suffix)]
    
    #Add the full Path
    filenames = [path_to_dir + v for v in filenames]
    
    return filenames

### Quality Audits
- QA Monitoring

In [177]:
DF_QA = "../Data/Phone Responses BPO.xlsx"
DF_QA = pd.read_excel(DF_QA, sheet_name="Raw")

In [178]:
DF_QA = DF_QA[DF_QA["Scope"] == "ATO"].reset_index(drop=True)

In [179]:
DF_QA = DF_QA[~(DF_QA["Score"].isnull())].reset_index(drop=True)

In [180]:
DF_QA["Score"] = DF_QA["Score"].astype(str).str.split("/").apply(lambda x: x[0]).astype(int)

#### Cleanup

In [181]:
DF_QA.columns = DF_QA.columns.str.strip()
DF_QA = DF_QA[~(DF_QA["Agent Reviewed (CB Email)"].isnull())].reset_index(drop=True)

In [182]:
DF_QA["Agent_Name_Cleaned"] = DF_QA["Agent Reviewed (CB Email)"].str.split("@").apply(lambda x: x[0])\
                                                                .str.split("_").apply(lambda x: x[0])

In [183]:
DF_QA["Agent_Name_Cleaned"] = \
DF_QA["Agent_Name_Cleaned"].apply(lambda x: " ".join([v for v in x.split(".") if len(v) > 1]))
DF_QA = DF_QA[~(DF_QA["Case Being QA'ed: Case Number"].isnull())].reset_index(drop=True)
DF_QA.rename(columns={"Case Being QA'ed: Case Number": "Case Number"}, inplace=True)
DF_QA["Case Number"] = DF_QA["Case Number"].astype(int)

DF_QA = DF_QA[~(DF_QA["Score"].isnull())].reset_index(drop=True)
DF_QA["Agent_Name_Cleaned"] = DF_QA["Agent_Name_Cleaned"].str.lower().str.strip()

DF_QA["Date of evaluation"] = pd.to_datetime(DF_QA["Date of evaluation"])
DF_QA["Evaluation_Month"] = DF_QA["Date of evaluation"].dt.month_name()

In [184]:
DF_QA["Evaluation_Week"] = DF_QA["Date of evaluation"].dt.to_period("W").apply(lambda r: r.start_time)

In [185]:
FEATS = \
["Did the agent transfer/escalate the case?", "Greeting",
 "Active Listening & Acknowledgement", "Customer Engagement", "Probing & Validation",
 "Resolution", "Closing & Recap", "Call Pace & Hold Procedure", "Case & Account Comments",
 "Case Labels, Workflow Picklist, Case Status", "With CE?", "Was the issue resolved?",
 "Did the Analyst able to complete phone authentication and verify caller?", "Did the Analyst avoid disclosing MNPI or private information?"
]

In [186]:
DF_QA[FEATS] = DF_QA[FEATS].replace({"No":0, "Yes":100})

### Scope

In [187]:
DF_QA = DF_QA[DF_QA["Evaluation_Week"] >= "11-08-2021"]

### Surveys

In [188]:
SURVEY_DIR = "../Data/"
TO_PROCESS = Locate_Files(SURVEY_DIR, (".csv"))

DF_SURVEY = []
for file in TO_PROCESS:
    a = pd.read_csv(file)
    DF_SURVEY.append(a)


DF_SURVEY = pd.concat(DF_SURVEY, axis=0)
DF_SURVEY.rename(columns={"Case: Case Number": "Case Number"}, inplace=True)

C:\Users\marcus.g.morante\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Cleanup

In [190]:
DF_SURVEY["Agent_Name_Cleaned"] = DF_SURVEY["Survey Response: Owner Name"].str.lower()

In [191]:
DF_SURVEY["Survey Response: Created Date"] = pd.to_datetime(DF_SURVEY["Survey Response: Created Date"])

In [192]:
DF_SURVEY["Survey_Month"] = DF_SURVEY["Survey Response: Created Date"].dt.month_name()
DF_SURVEY["Resolution"] = DF_SURVEY["Was your issue resolved?"].apply(lambda x: 100 if "Yes" in x else 0)

In [193]:
DF_SURVEY["Resolution"] = DF_SURVEY["Was your issue resolved?"].apply(lambda x: 100 if "Yes" in x else 0)

In [194]:
DF_SURVEY["Survey Week"] = DF_SURVEY["Survey Response: Created Date"].dt.to_period("W").apply(lambda r: r.start_time)

In [195]:
DF_SURVEY = DF_SURVEY[DF_SURVEY["Survey Response: Created Date"] >= "11-08-2021"].reset_index(drop=True)

In [196]:
DF_SURVEY.groupby([DF_SURVEY["Survey Response: Created Date"].dt.month,DF_SURVEY["Survey Response: Created Date"].dt.day])["Case Number"].count().unstack()

Survey Response: Created Date,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
Survey Response: Created Date,,,,,,,,,,,,,,,,,,,,,
1,510.0,430.0,416.0,575.0,687.0,678.0,685.0,631.0,531.0,547.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,965.0,1263.0,1342.0,...,674.0,871.0,732.0,685.0,578.0,528.0,519.0,647.0,768.0,NaN
12,768.0,765.0,741.0,657.0,487.0,571.0,702.0,752.0,575.0,586.0,...,515.0,508.0,534.0,525.0,417.0,430.0,565.0,638.0,736.0,651.0


# Data Engineering

## Non-Merged Data

In [197]:
QUERY = DF_SURVEY.groupby(["Agent_Name_Cleaned"])["Resolution"].mean().round(2).reset_index()

In [198]:
quartiles = QUERY["Resolution"].quantile([.25, .5, .75]).tolist()
quartiles = [0] + quartiles + [float('inf')]

In [199]:
QUERY["Survey_Quartile"] = \
pd.cut(QUERY["Resolution"], bins=quartiles, labels=['1st','2nd', '3rd', '4th'])

In [200]:
QUERY_L2_Reso = \
DF_SURVEY.groupby(["Agent_Name_Cleaned","Case: L2 Workflow"])["Resolution"].mean().unstack().reset_index()

QUERY_L2_Count = \
DF_SURVEY.groupby(["Agent_Name_Cleaned","Case: L2 Workflow"])["Resolution"].count().unstack().reset_index()

In [201]:
QUERY_QA = DF_QA.groupby(["Agent_Name_Cleaned"])["Score"].mean().round(2).reset_index()

In [202]:
quartiles = QUERY_QA["Score"].quantile([.25, .5, .75]).tolist()
quartiles = [0] + quartiles + [float('inf')]

In [203]:
QUERY_QA["QA_Quartile"] = \
pd.cut(QUERY_QA["Score"], bins=quartiles, labels=['1st','2nd', '3rd', '4th'])

In [204]:
QUERY_QA.head()

,Agent_Name_Cleaned,Score,QA_Quartile
0,abegail puerta,97.30,3rd
1,abelardo gomezjr,87.08,2nd
2,agnes mae morales,98.48,4th
3,agustin bagguatan,97.86,4th
4,alavinia abdah,100.00,4th


In [205]:
QUERY = \
QUERY.merge(QUERY_QA, on="Agent_Name_Cleaned", how="left")\
     .merge(QUERY_L2_Reso, on="Agent_Name_Cleaned", how="left")\
     .merge(QUERY_L2_Count, on="Agent_Name_Cleaned", how="left", suffixes=("_Reso","_Count"))

## Merged Data

In [206]:
DF_CONSOLIDATED = \
DF_QA.merge(DF_SURVEY.drop(columns=["Agent_Name_Cleaned"]).rename(columns={"Resolution": "Resolution Survey"}), on = "Case Number", how = "left")

DF_CONSOLIDATED = \
DF_CONSOLIDATED[~(DF_CONSOLIDATED["Resolution Survey"].isnull())].reset_index(drop=True)

In [207]:
QUERY_1 = DF_CONSOLIDATED.groupby(["Agent_Name_Cleaned"])["Resolution Survey"].mean().round(2).reset_index()

In [208]:
quartiles = QUERY_1["Resolution Survey"].quantile([.25, .5, .75]).tolist()
quartiles = quartiles + [float('inf')]
quartiles

[0.0, 25.0, 45.45, inf]

In [209]:
QUERY_1["Survey_Quartile"] = \
pd.cut(QUERY_1["Resolution Survey"], bins=quartiles, labels=['2nd', '3rd', '4th'])

QUERY_1["Survey_Quartile"] = QUERY_1["Survey_Quartile"].cat.add_categories("1st")
QUERY_1["Survey_Quartile"].fillna("1st", inplace =True) 

In [210]:
QUERY_L2_Reso_1 = \
DF_CONSOLIDATED.groupby(["Agent_Name_Cleaned","Case: L2 Workflow"])["Resolution"].mean().unstack().reset_index()

QUERY_L2_Count_1 = \
DF_CONSOLIDATED.groupby(["Agent_Name_Cleaned","Case: L2 Workflow"])["Resolution"].count().unstack().reset_index()

In [211]:
QUERY_QA_1 = DF_CONSOLIDATED.groupby(["Agent_Name_Cleaned"])["Score"].mean().round(2).reset_index()

In [212]:
quartiles = QUERY_QA_1["Score"].quantile([.25, .5, .75]).tolist()
quartiles = [0] + quartiles + [float('inf')]
quartiles

[0, 88.33, 95.0, 100.0, inf]

In [213]:
QUERY_QA_1["QA_Quartile"] = \
pd.cut(QUERY_QA_1["Score"], bins=quartiles, labels=['1st','2nd', '3rd', '4th'])

In [214]:
QUERY_1 = \
QUERY_1.merge(QUERY_QA_1, on="Agent_Name_Cleaned", how="left")\
       .merge(QUERY_L2_Reso_1, on="Agent_Name_Cleaned", how="left")\
       .merge(QUERY_L2_Count_1, on="Agent_Name_Cleaned", how="left", suffixes=("_Reso","_Count"))

In [215]:
#QUERY_1.to_excel("Merge_Data.xlsx", index=False)

## Get Schedule and Supervisor

In [216]:
DF_SCHED = pd.read_csv("../Data/Other Details/Agent Schedules.csv")

In [217]:
DF_SCHED["Agent_Name_Cleaned"] = DF_SCHED["Enterprise ID"].str.split(".").apply(lambda x: x[0] + " " + x[-1]).str.lower()

In [218]:
col_list = ["W1-Sunday", "W2-Monday", "W2-Tuesday", "W2-Wednesday", "W2-Thursday", "W2-Friday", "W2-Saturday", "W2-Sunday"]
DF_SCHED[col_list] = DF_SCHED[col_list].replace("OFF", np.nan)
DF_SCHED[col_list] = DF_SCHED[col_list].replace("ML", np.nan)
DF_SCHED[col_list] = DF_SCHED[col_list].replace("VL", np.nan)

In [219]:
for col in col_list:
    DF_SCHED[col] = DF_SCHED[col].str.replace("|", "-", regex=True)
    DF_SCHED[col] = DF_SCHED[col].str.replace("^", "-", regex=True)
    DF_SCHED[col] = DF_SCHED[col].str.replace(":", "-", regex=True)
    DF_SCHED[col] = DF_SCHED[col].str.replace("~", "-", regex=True)

In [220]:
DF_SCHED["Schedule_Ref"] = DF_SCHED["W2-Monday"].fillna(DF_SCHED["W2-Tuesday"])
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].fillna(DF_SCHED["W2-Wednesday"])
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].fillna(DF_SCHED["W2-Thursday"])
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].fillna(DF_SCHED["W2-Friday"])
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].fillna(DF_SCHED["W2-Saturday"])
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].fillna(DF_SCHED["W2-Sunday"])

In [221]:
DF_SCHED = DF_SCHED[~(DF_SCHED["Schedule_Ref"].isnull())].reset_index(drop=True)

In [222]:
DF_SCHED["Schedule_Ref"] = DF_SCHED["Schedule_Ref"].str.split("-").apply(lambda x: x[0] + ":" + x[1]).apply(pd.to_datetime)

In [223]:
TO_CHECK = DF_SCHED["Schedule_Ref"].dt.hour

CONDITIONS = [(TO_CHECK >= 6) & (TO_CHECK < 13),
              (TO_CHECK >= 13) & (TO_CHECK < 17)
             ]

CHOICES = ["Morning", "Mid"]

DF_SCHED["Shift Schedule"] = np.select(CONDITIONS, CHOICES, "Night")

In [224]:
DF_SCHED = DF_SCHED[["Agent_Name_Cleaned", "Direct Supervisor", "Shift Schedule"]]

In [225]:
QUERY = QUERY.merge(DF_SCHED, on="Agent_Name_Cleaned",how="left")
QUERY_1 = QUERY_1.merge(DF_SCHED, on="Agent_Name_Cleaned",how="left")

In [226]:
QUERY.shape

(711, 79)

In [227]:
QUERY_1.shape

(101, 25)

## Get Tenure & Wave

In [228]:
DF_TENURE = pd.read_excel("../Data/Other Details/Accenture Roster.xlsx", sheet_name="Roster", skiprows=4)

In [229]:
DF_TENURE = DF_TENURE[~(DF_TENURE["CoinbaseEmail"].replace("-",np.nan).isnull())].reset_index(drop=True)

In [230]:
DF_TENURE["Agent_Name_Cleaned"] = DF_TENURE["CoinbaseEmail"].str.split("@").apply(lambda x: x[0].split(".")[:-1])
DF_TENURE["Agent_Name_Cleaned"] = DF_TENURE["Agent_Name_Cleaned"].apply(lambda x: x[0] + " " + x[-1].split("_")[0].replace("external","")).str.lower()

In [231]:
DF_TENURE["Tenure in Pillar"] = DF_TENURE["Tenure in Pillar"].fillna(DF_TENURE["Tenure"])

In [232]:
DF_TENURE = \
DF_TENURE[["Agent_Name_Cleaned", "Tenure in Pillar", "Latest Training Wave"]]

In [233]:
DF_TENURE["Tenure in Pillar"].value_counts()

>90d      1101
31-60d      55
61-90d      44
Name: Tenure in Pillar, dtype: int64

In [234]:
QUERY = QUERY.merge(DF_TENURE, on="Agent_Name_Cleaned",how="left")
QUERY_1 = QUERY_1.merge(DF_TENURE, on="Agent_Name_Cleaned",how="left")

In [235]:
QUERY.shape

(711, 81)

In [236]:
QUERY_1.shape

(101, 27)

## Coaching

In [237]:
DF_COACHING = pd.read_csv("../Data/Other Details/ATO QA Dashboard_ATO QA_Table.csv")

In [238]:
DF_COACHING["Completion%"] = DF_COACHING["Completion%"].replace("-", np.nan).astype(float).round(2)
DF_COACHING["Compliance%"] = DF_COACHING["Compliance%"].replace("-", np.nan).astype(float).round(2)

In [239]:
DF_COACHING

,Agent Name,QA%,CE,Completion%,Compliance%
0,"Nalayog, Ma. Elvie",1.000000,0,NaN,NaN
1,"Abdah, Alavinia",1.000000,0,NaN,NaN
2,"Lachica, Nathalia",0.997000,0,1.00,0.50
3,"Juego, Heherson",0.996226,0,1.00,0.00
4,"Abanil, Joy",0.995455,0,1.00,2.00
...,...,...,...,...,...
89,"Mangahas, Jackielyn",0.764865,2,0.31,0.46
90,"Veldosola, Anecito",0.740000,4,0.33,0.28
91,"Siochi, John Lidnel",0.732692,3,0.25,0.31
92,"Irlandez, Charles",0.658000,3,0.29,0.24


In [240]:
DF_COACHING["Agent_Name_Cleaned"] = \
DF_COACHING["Agent Name"].str.split(",").apply(lambda x: x[-1] + " " + x[0]).str.strip().str.lower()

In [241]:
DF_COACHING = DF_COACHING[["Agent_Name_Cleaned", "Completion%", "Compliance%"]]

In [242]:
QUERY = QUERY.merge(DF_COACHING, on="Agent_Name_Cleaned",how="left")
QUERY_1 = QUERY_1.merge(DF_COACHING, on="Agent_Name_Cleaned",how="left")

In [243]:
QUERY.head()

,Agent_Name_Cleaned,Resolution,Survey_Quartile,Score,QA_Quartile,Account Access_Reso,Account Safety_Reso,Account Risk_Reso,Crypto Ops_Reso,Wallet_Reso,...,Coinbase One_Count,Payment Ops_Count,Save & Spend_Count,Facematch_Count,Direct Supervisor,Shift Schedule,Tenure in Pillar,Latest Training Wave,Completion%,Compliance%
0,abagael eulogio,42.86,4th,NaN,NaN,0.0,44.444444,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ynah.v.o.baltazar,Night,NaN,NaN,NaN,NaN
1,abdul villalon,28.57,3rd,NaN,NaN,NaN,NaN,25.0,37.5,0.0,...,NaN,NaN,NaN,NaN,Bryan.c.entera,Morning,NaN,NaN,NaN,NaN
2,abegail puerta,44.90,4th,97.3,3rd,50.0,44.186047,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,jasmine.t.alcantara,Night,>90d,Voice ATO 1 - T2 Access A,0.67,0.67
3,adrian umandap,33.33,3rd,NaN,NaN,NaN,NaN,100.0,0.0,NaN,...,NaN,NaN,NaN,NaN,althea.secretaria,Morning,NaN,NaN,NaN,NaN
4,agnes morales,36.84,4th,NaN,NaN,0.0,41.176471,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,>90d,Voice ATO 2 - T2 Access C,NaN,NaN


In [244]:
QUERY_1.head()

,Agent_Name_Cleaned,Resolution Survey,Survey_Quartile,Score,QA_Quartile,ACH_Reso,Account Access_Reso,Account Risk_Reso,Account Safety_Reso,Compliance Policy_Reso,...,Crypto Ops_Count,Other_Count,Pro_Count,Wallet_Count,Direct Supervisor,Shift Schedule,Tenure in Pillar,Latest Training Wave,Completion%,Compliance%
0,abegail puerta,45.45,3rd,96.36,3rd,NaN,NaN,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,jasmine.t.alcantara,Night,>90d,Voice ATO 1 - T2 Access A,0.67,0.67
1,abelardo gomezjr,0.00,1st,95.00,2nd,NaN,NaN,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,agnes mae morales,0.00,1st,100.00,3rd,NaN,NaN,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,1.00
3,agustin bagguatan,0.00,1st,88.33,1st,NaN,NaN,100.0,50.0,NaN,...,NaN,NaN,NaN,NaN,eddelyn.v.t.bendana,Morning,>90d,Voice ATO 6,NaN,NaN
4,amelia zuniga,0.00,1st,87.50,1st,NaN,NaN,NaN,50.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [245]:
QUERY.to_excel("No-Merge_Data.xlsx", index=False)
QUERY_1.to_excel("Merge_Data.xlsx", index=False)

## Creating Week on Week Workflow Behavior

In [151]:
selected_workflows = \
["Retail Onboarding", "Account Safety", "Account Access", "Compliance Policy",
 "Account Risk", "ACH", "Crypto Ops", "Cards", "SEPA", "App Content & Community", "Wallet"]

In [152]:
GET_DATA = \
DF_SURVEY.groupby(["Survey Week", "Case: L2 Workflow"],as_index=False)["Resolution"].agg(["mean","count"]).reset_index()

In [153]:
GET_DATA = GET_DATA[GET_DATA["Case: L2 Workflow"].isin(selected_workflows)].reset_index(drop=True)

In [154]:
GET_DATA.to_excel("Sheet3.xlsx", index=False)

## For Sheet 2

In [155]:
DF_CONSOLIDATED[["Case Number", "Score", "Resolution Survey",
                 "Case: L1 Workflow", "Case: L2 Workflow", "Case: L3 Workflow", "Case: Last Macro Applied", "Was the issue resolved?"]].to_excel("Sheet2.xlsx", index=False)

## Other Items

In [159]:
DF_CONSOLIDATED.head()

,Timestamp,Email Address,Score,Start Time,Scope,Date of evaluation,Agent Reviewed (CB Email),Case Number,Customer Talkdesk URL,Call Duration,...,Case: Case Agent Handle Time (Hours),Case: Sub Team,Case: Team,Survey Response: Owner Name,Case: Case Owner Email,How can we do better?,What did we do well?,Survey_Month,Resolution Survey,Survey Week
0,2021-11-08 19:19:11.864,jimbo.hilaga_external.acn@coinbase.com,95,19:00:00,ATO,2021-11-08,romalyn.ortazo@coinbase.com,8361429,https://coinbase.mytalkdesk.com/#contacts/6183...,00:09:30,...,0.55,Safety,Account Ops,Noelito Bulatao,noelito.bulatao@coinbase.com,NaN,NaN,November,0.0,2021-11-15
1,2021-11-09 00:49:19.387,jimbo.hilaga_external.acn@coinbase.com,100,00:30:00,ATO,2021-11-08,agustin.b.bagguatan_external.acn@coinbase.com,8387904,https://coinbase.mytalkdesk.com/#contacts/615a...,00:09:06,...,386.53,Risk,Account Ops,Agustin Bagguatan,rayann.gabalones@coinbase.com,NaN,NaN,November,0.0,2021-11-08
2,2021-11-09 01:53:54.352,jimbo.hilaga_external.acn@coinbase.com,100,01:44:00,ATO,2021-11-08,anecito.veldosola@coinbase.com,8385092,https://coinbase.mytalkdesk.com/#contacts/6187...,00:05:22,...,0.07,Other,Other,Anecito Veldosola,anecito.veldosola@coinbase.com,NaN,NaN,November,0.0,2021-11-08
3,2021-11-09 06:39:40.251,arey.mahusay@coinbase.com,100,02:28:00,ATO,2021-11-09,bernard.j.t.eugenio_external.acn@coinbase.com,8385836,https://coinbase.mytalkdesk.com/#contacts/6188...,00:07:03,...,0.14,Other,Other,bernard eugenio,eleonor.baniqued_external.acn@coinbase.com,NaN,NaN,November,0.0,2021-11-08
4,2021-11-09 07:33:15.938,clarez.courts_external.acn@coinbase.com,75,07:21:00,ATO,2021-11-09,chad.cledera@coinbase.com,8403703,https://coinbase.mytalkdesk.com/#contacts/6184...,00:08:42,...,0.15,Other,Other,Chad Cledera,chad.cledera@coinbase.com,NaN,NaN,November,0.0,2021-11-08


In [160]:
DF_SURVEY_MODIFIED = \
DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(DF_CONSOLIDATED["Agent_Name_Cleaned"].to_list())].reset_index(drop=True)

In [161]:
DF_SURVEY_MODIFIED.head()

,Case Number,Was your issue resolved?,Additional Feedback,Case: Submitted Category,Case: Category,Case: Subcategory,Case: Last Macro Applied,Survey Response: Created Date,Case: Survey Sent Date/Time,Case: Date/Time Opened,...,Case: Sub Team,Case: Team,Survey Response: Owner Name,Case: Case Owner Email,How can we do better?,What did we do well?,Agent_Name_Cleaned,Survey_Month,Resolution,Survey Week
0,8837336,No,I still can access my portfolio,NaN,Account,NaN,Pro > Managing Portfolios,2021-12-14,12/14/2021 8:00 AM,12/1/2021 7:22 AM,...,Safety,Account Ops,Ma Elvie Nalayog,harvey.albino_external.acn@coinbase.com,NaN,NaN,ma elvie nalayog,December,0,2021-12-13
1,8964559,No,Please see my increasing frustration in the em...,vault,Fraud,Banned User,Fraud Unban- Pending Review,2021-12-09,12/9/2021 7:30 AM,12/8/2021 8:43 AM,...,Risk,Account Ops,kathrine castillo,NaN,NaN,NaN,kathrine castillo,December,0,2021-12-06
2,9274952,Yes,NaN,NaN,Compliance,NaN,Happy Place - Thank You,2021-12-29,12/28/2021 4:30 PM,12/27/2021 3:59 PM,...,Compliance,Policy Ops,Maricel Osera,monika.more@coinbase.com,NaN,NaN,maricel osera,December,100,2021-12-27
3,9271902,Yes,I appreciate Coinbase’s attempts to rapidly de...,NaN,Account,NaN,Happy Place - Thank You,2021-12-28,12/28/2021 1:30 PM,12/27/2021 1:10 PM,...,Access,Account Ops,Julie Loyola,red.albaladejo@coinbase.com,NaN,NaN,julie loyola,December,100,2021-12-27
4,9239427,No,Still can not reset password and have not rece...,NaN,Account Compromise,NaN,Duplicate Case - Repeat Request,2021-12-27,12/26/2021 6:00 AM,12/25/2021 5:28 AM,...,Safety,Account Ops,Myra Gorospe,maribel.alkonga@coinbase.com,NaN,NaN,myra gorospe,December,0,2021-12-27


### Actual resolution rate % and distribution per quartile

In [162]:
QUERY = DF_SURVEY_MODIFIED.groupby(["Agent_Name_Cleaned"])["Resolution"].mean().round(2).reset_index()

In [163]:
quartiles = QUERY["Resolution"].quantile([.25, .5, .75]).tolist()
quartiles = [0] + quartiles + [float('inf')]

QUERY["Resolution_Quartile"] = \
pd.cut(QUERY["Resolution"], bins=quartiles, labels=['1st','2nd', '3rd', '4th'])

In [164]:
RESULT = \
pd.concat([QUERY["Resolution_Quartile"].value_counts().reindex(["1st", "2nd", "3rd", "4th"]).rename("Count"),
           QUERY["Resolution_Quartile"].value_counts(normalize=True).reindex(["1st", "2nd", "3rd", "4th"]).rename("% Count"),
           QUERY.groupby(["Resolution_Quartile"])["Resolution"].mean().rename("Resolution %")
          ],axis=1)

RESULT["Resolution %"] = RESULT["Resolution %"].round(2)
RESULT

,Count,% Count,Resolution %
1st,22,0.261905,21.07
2nd,20,0.238095,29.73
3rd,22,0.261905,34.80
4th,20,0.238095,47.75


### ANOVA on each of the workflows

In [165]:
DF_SURVEY_MODIFIED["Case: L2 Workflow"].value_counts()

Account Safety              2260
Account Access               250
Account Risk                 120
Other                         27
Compliance Policy             17
Crypto Ops                    17
ACH                            6
Wallet                         6
Pro                            5
API Support                    3
Retail Onboarding              3
Scaled Developer Support       1
Financing                      1
SEPA                           1
Name: Case: L2 Workflow, dtype: int64

In [521]:
ANNOV_A = DF_SURVEY_MODIFIED[DF_SURVEY_MODIFIED["Case: L2 Workflow"] == "Account Safety"]["Resolution"].rename("Account Safety")
ANNOV_B = DF_SURVEY_MODIFIED[DF_SURVEY_MODIFIED["Case: L2 Workflow"] == "Account Access"]["Resolution"].rename("Account Access")
ANNOV_C = DF_SURVEY_MODIFIED[DF_SURVEY_MODIFIED["Case: L2 Workflow"] == "Account Risk"]["Resolution"].rename("Account Risk")

In [522]:
from scipy.stats import f_oneway

In [523]:
f_oneway(ANNOV_A, ANNOV_B, ANNOV_C)

F_onewayResult(statistic=4.237278384163779, pvalue=0.014545720759385823)

In [526]:
#For Anova
pd.concat([ANNOV_A.reset_index(drop=True), ANNOV_B.reset_index(drop=True), ANNOV_C.reset_index(drop=True)],axis=1).to_excel("For Anova.xlsx", index=False)

In [512]:
pd.DataFrame([ANNOV_A, ANNOV_B, ANNOV_C])

,1,3,4,5,6,7,8,9,10,11,...,2730,2731,2732,2733,2734,2735,2736,2737,2738,2739
Resolution,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
Resolution,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
Resolution,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [169]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [171]:
SAMPLE = pd.DataFrame()
SAMPLE["Account_Safety"] = ANNOV_A.reset_index(drop=True)
SAMPLE["Account_Access"] = ANNOV_B.reset_index(drop=True)
SAMPLE["Account_Risk"] = ANNOV_C.reset_index(drop=True)

In [172]:
model = ols("Account_Safety ~ Account_Risk + Account_Access + Account_Risk:Account_Access",data = SAMPLE).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

In [173]:
anova_table

,sum_sq,df,F,PR(>F)
Account_Risk,302.997705,1.0,0.126617,0.722612
Account_Access,22.541093,1.0,0.009419,0.922851
Account_Risk:Account_Access,11980.965880,1.0,5.006617,0.027161
Residual,277591.036415,116.0,NaN,NaN


- Row 1 & 2: Against **Safety** 
- Row 3: **Risk** vs **Access**

### Shift Schedule using Case Closed

In [252]:
DF_SURVEY_MODIFIED["Case: Date/Time Closed"] = DF_SURVEY_MODIFIED["Case: Date/Time Closed"].apply(pd.to_datetime)

In [257]:
NEW_QUERY = \
DF_SURVEY_MODIFIED.copy()

In [262]:
NEW_QUERY = NEW_QUERY[~(NEW_QUERY["Case: Date/Time Closed"].isnull())].reset_index(drop=True)

In [336]:
TO_CHECK = NEW_QUERY["Case: Date/Time Closed"].dt.hour

CONDITIONS = [(TO_CHECK >= 6) & (TO_CHECK < 13),
              (TO_CHECK >= 13) & (TO_CHECK < 17)
             ]

CHOICES = ["Morning", "Mid"]

NEW_QUERY["Shift Schedule"] = np.select(CONDITIONS, CHOICES, "Night")

In [347]:
TO_CHECK = NEW_QUERY["Case: Date/Time Closed"].dt.hour

CONDITIONS = [(TO_CHECK >= 6) & (TO_CHECK < 15),
              (TO_CHECK >= 15) & (TO_CHECK < 23)
             ]

CHOICES = ["Morning", "Mid"]

NEW_QUERY["Shift Schedule"] = np.select(CONDITIONS, CHOICES, "Night")

In [348]:
NEW_QUERY.groupby(["Shift Schedule", NEW_QUERY["Case: Date/Time Closed"].dt.month])["Resolution"].count().reset_index()

,Shift Schedule,Case: Date/Time Closed,Resolution
0,Mid,1,33
1,Mid,11,270
2,Mid,12,1362
3,Morning,11,42
4,Morning,12,61
5,Night,11,26
6,Night,12,22


In [349]:
NEW_QUERY.groupby(["Shift Schedule", NEW_QUERY["Case: Date/Time Closed"].dt.month])["Resolution"].mean().reset_index()

,Shift Schedule,Case: Date/Time Closed,Resolution
0,Mid,1,39.393939
1,Mid,11,29.629630
2,Mid,12,35.022026
3,Morning,11,9.523810
4,Morning,12,6.557377
5,Night,11,19.230769
6,Night,12,31.818182


In [350]:
NEW_QUERY.groupby(["Shift Schedule", NEW_QUERY["Case: Date/Time Closed"].dt.month])["Agent_Name_Cleaned"].nunique().reset_index()

,Shift Schedule,Case: Date/Time Closed,Agent_Name_Cleaned
0,Mid,1,25
1,Mid,11,76
2,Mid,12,83
3,Morning,11,26
4,Morning,12,42
5,Night,11,19
6,Night,12,19


In [351]:
pd.concat([NEW_QUERY.groupby(["Shift Schedule"])["Resolution"].count().rename("Count"),
           NEW_QUERY.groupby(["Shift Schedule"])["Resolution"].mean().round(2).rename("% Resolution"),
           NEW_QUERY.groupby(["Shift Schedule"])["Agent_Name_Cleaned"].nunique().rename("Agent Count")
          ], axis=1)

,Count,% Resolution,Agent Count
Shift Schedule,,,
Mid,1665,34.23,84
Morning,103,7.77,55
Night,48,25.00,33


In [283]:
MERGED_DATA = \
DF_QA.merge(DF_SURVEY.drop(columns="Agent_Name_Cleaned").rename(columns={"Resolution": "Resolution_Survey"}), on="Case Number")

In [288]:
MERGED_DATA["Agent_Name_Cleaned"].nunique()

101

In [535]:
all_cols = \
["Greeting", "Active Listening & Acknowledgement", "Customer Engagement", "Probing & Validation", "Resolution",
 "Closing & Recap", "Call Pace & Hold Procedure", "Case & Account Comments", "Case Labels, Workflow Picklist, Case Status",
 "With CE?", "Did the Analyst able to complete phone authentication and verify caller?",
 "Did the Analyst avoid disclosing MNPI or private information?", "Score", "Resolution_Survey"]

In [689]:
MERGED_DATA["Survey Week"].min()

Timestamp('2021-11-08 00:00:00')

In [690]:
MERGED_DATA["Survey Week"].max()

Timestamp('2022-01-17 00:00:00')

In [300]:
AGENT_PERFORMANCE = \
MERGED_DATA.groupby(["Agent_Name_Cleaned"])[all_cols].mean()

In [352]:
AGENT_PERFORMANCE.round(2)

,Greeting,Active Listening & Acknowledgement,Customer Engagement,Probing & Validation,Resolution,Closing & Recap,Call Pace & Hold Procedure,Case & Account Comments,"Case Labels, Workflow Picklist, Case Status",With CE?,Did the Analyst able to complete phone authentication and verify caller?,Did the Analyst avoid disclosing MNPI or private information?,Score,Resolution_Survey,Survey_Quartile,QAMonitoring_Quartile
Agent_Name_Cleaned,,,,,,,,,,,,,,,,
abegail puerta,100.0,90.91,90.91,100.0,100.00,90.91,100.0,100.0,100.0,0.0,100.0,100.0,96.36,45.45,3rd,3rd
abelardo gomezjr,100.0,100.00,100.00,100.0,100.00,100.00,0.0,100.0,100.0,0.0,100.0,100.0,95.00,0.00,1st,2nd
agnes mae morales,100.0,100.00,100.00,100.0,100.00,100.00,100.0,100.0,100.0,0.0,100.0,100.0,100.00,0.00,1st,3rd
agustin bagguatan,100.0,100.00,100.00,100.0,66.67,66.67,100.0,100.0,100.0,0.0,100.0,100.0,88.33,0.00,1st,1st
amelia zuniga,100.0,100.00,100.00,100.0,50.00,100.00,100.0,100.0,100.0,50.0,100.0,100.0,87.50,0.00,1st,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
veronica badillo,100.0,100.00,100.00,0.0,0.00,100.00,100.0,0.0,0.0,100.0,100.0,100.0,50.00,100.00,4th,1st
victoria badillo,100.0,0.00,100.00,0.0,100.00,100.00,100.0,100.0,100.0,0.0,100.0,100.0,70.00,0.00,1st,1st
virginia leonin,100.0,100.00,100.00,100.0,100.00,100.00,100.0,100.0,100.0,0.0,100.0,100.0,100.00,25.00,2nd,3rd


In [311]:
quartiles = AGENT_PERFORMANCE["Resolution_Survey"].quantile([.25, .5, .75]).tolist()
quartiles = quartiles + [float('inf')]
print(quartiles)

AGENT_PERFORMANCE["Survey_Quartile"] = \
pd.cut(AGENT_PERFORMANCE["Resolution_Survey"], bins=quartiles, labels=['2nd', '3rd', '4th'])

AGENT_PERFORMANCE["Survey_Quartile"] = AGENT_PERFORMANCE["Survey_Quartile"].cat.add_categories("1st")
AGENT_PERFORMANCE["Survey_Quartile"].fillna("1st", inplace =True) 

[0.0, 25.0, 45.45454545454545, inf]


In [312]:
quartiles = AGENT_PERFORMANCE["Score"].quantile([.25, .5, .75]).tolist()
quartiles = [0] + quartiles + [float('inf')]
print(quartiles)

AGENT_PERFORMANCE["QAMonitoring_Quartile"] = \
pd.cut(AGENT_PERFORMANCE["Score"], bins=quartiles, labels=['1st','2nd', '3rd', '4th'])

[0, 88.33333333333333, 95.0, 100.0, inf]


In [551]:
AGENT_PERFORMANCE.groupby(["Survey_Quartile"])["Resolution_Survey"].count().reindex(['1st','2nd', '3rd', '4th']).round(2)

Survey_Quartile
1st    33
2nd    21
3rd    22
4th    25
Name: Resolution_Survey, dtype: int64

In [562]:
First_Quarts = AGENT_PERFORMANCE[AGENT_PERFORMANCE["Survey_Quartile"] == "1st"].index.to_list()

In [603]:
pd.concat([AGENT_PERFORMANCE.groupby(["Survey_Quartile"])[all_cols].mean().reindex(['1st','2nd', '3rd', '4th']).round(2),
           AGENT_PERFORMANCE.groupby(["Survey_Quartile"])["Resolution_Survey"].count().reindex(['1st','2nd', '3rd', '4th']).rename("Count")],
          axis=1
         )

,Greeting,Active Listening & Acknowledgement,Customer Engagement,Probing & Validation,Resolution,Closing & Recap,Call Pace & Hold Procedure,Case & Account Comments,"Case Labels, Workflow Picklist, Case Status",With CE?,Did the Analyst able to complete phone authentication and verify caller?,Did the Analyst avoid disclosing MNPI or private information?,Score,Resolution_Survey,Count
Survey_Quartile,,,,,,,,,,,,,,,
1st,100.0,96.97,85.86,87.27,90.05,92.17,87.37,91.06,90.91,3.03,98.48,100.00,90.71,0.00,33
2nd,100.0,91.98,88.58,87.51,90.33,94.02,89.61,92.98,92.14,3.64,97.15,99.57,90.93,19.40,21
3rd,100.0,99.17,94.70,90.23,89.33,94.04,91.29,91.81,91.36,3.63,98.59,99.55,93.08,36.47,22
4th,96.0,100.00,97.13,90.93,93.13,93.87,90.33,85.40,87.23,8.80,100.00,96.00,93.83,74.93,25


## Chi-Square

In [545]:
CHI2_TEST = \
MERGED_DATA[all_cols]

CHI2_TEST = CHI2_TEST + 1

In [546]:
from scipy.stats import chisquare

In [547]:
all_CHI = \
["Greeting", "Active Listening & Acknowledgement", "Customer Engagement", "Probing & Validation", "Resolution",
 "Closing & Recap", "Call Pace & Hold Procedure", "Case & Account Comments", "Case Labels, Workflow Picklist, Case Status",
 "With CE?", "Did the Analyst able to complete phone authentication and verify caller?",
 "Did the Analyst avoid disclosing MNPI or private information?", "Score"]

#"Resolution_Survey"

In [548]:
for feat in all_CHI:
    print(f"{feat} vs. Resolution Survey ")
    print(chisquare(CHI2_TEST[feat], CHI2_TEST["Resolution_Survey"]),"\n")

Greeting vs. Resolution Survey 
Power_divergenceResult(statistic=3080099.00990099, pvalue=0.0) 

Active Listening & Acknowledgement vs. Resolution Survey 
Power_divergenceResult(statistic=2960099.00990099, pvalue=0.0) 

Customer Engagement vs. Resolution Survey 
Power_divergenceResult(statistic=2750198.0198019804, pvalue=0.0) 

Probing & Validation vs. Resolution Survey 
Power_divergenceResult(statistic=2731089.108910891, pvalue=0.0) 

Resolution vs. Resolution Survey 
Power_divergenceResult(statistic=2730594.0594059406, pvalue=0.0) 

Closing & Recap vs. Resolution Survey 
Power_divergenceResult(statistic=2870594.0594059406, pvalue=0.0) 

Call Pace & Hold Procedure vs. Resolution Survey 
Power_divergenceResult(statistic=2761287.1287128716, pvalue=0.0) 

Case & Account Comments vs. Resolution Survey 
Power_divergenceResult(statistic=2851188.118811881, pvalue=0.0) 

Case Labels, Workflow Picklist, Case Status vs. Resolution Survey 
Power_divergenceResult(statistic=2851089.108910891, pval

## Prove Grouping

In [573]:
MERGED_DATA[MERGED_DATA["Agent_Name_Cleaned"].isin(First_Quarts)]["Agent_Name_Cleaned"].nunique()

33

In [568]:
MERGED_DATA[MERGED_DATA["Agent_Name_Cleaned"].isin(First_Quarts)]["Resolution_Survey"].value_counts()

0    74
Name: Resolution_Survey, dtype: int64

In [581]:
MERGED_DATA[MERGED_DATA["Agent_Name_Cleaned"].isin(First_Quarts)].groupby(["Agent_Name_Cleaned"])["Resolution_Survey"].value_counts().unstack().sort_values(by=0,ascending=False)

Resolution_Survey,0
Agent_Name_Cleaned,
myra gorospe,6
joy abanil,6
kevin manayon,6
jestony pujeda,5
charles irlandez,4
sharlann blanca,4
ella canaveral,3
paquito costuna,3
agnes mae morales,3


## Quartile Comparison

In [592]:
Survey_4th = AGENT_PERFORMANCE[AGENT_PERFORMANCE["Survey_Quartile"] == "4th"].index.to_series().drop_duplicates().to_list()
Survey_3rd = AGENT_PERFORMANCE[AGENT_PERFORMANCE["Survey_Quartile"] == "3rd"].index.to_series().drop_duplicates().to_list()
Survey_2nd = AGENT_PERFORMANCE[AGENT_PERFORMANCE["Survey_Quartile"] == "2nd"].index.to_series().drop_duplicates().to_list()
Survey_1st = AGENT_PERFORMANCE[AGENT_PERFORMANCE["Survey_Quartile"] == "1st"].index.to_series().drop_duplicates().to_list()

### Pillar

#### 4th

In [593]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: Team"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: Team"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)].groupby(["Case: Team"])["Resolution"].mean()], axis=1)

,count,count,Resolution
Account Ops,449,0.794690,35.634744
Other,98,0.173451,31.632653
BPO Ops,9,0.015929,22.222222
Policy Ops,4,0.007080,50.000000
Support - Payments,2,0.003540,50.000000
Client Services,2,0.003540,0.000000
Retail Ops,1,0.001770,0.000000


#### 1st

In [594]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: Team"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: Team"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)].groupby(["Case: Team"])["Resolution"].mean()], axis=1)

,count,count,Resolution
Account Ops,520,0.846906,27.884615
Other,70,0.114007,38.571429
BPO Ops,11,0.017915,9.090909
Policy Ops,6,0.009772,0.000000
Support - Wallet,3,0.004886,0.000000
Client Services,2,0.003257,50.000000
Retail Ops,2,0.003257,0.000000


### L2 Workflow

#### 4th

In [607]:
DF_SURVEY["Survey Week"].min()

Timestamp('2021-11-08 00:00:00')

In [606]:
DF_SURVEY["Survey Week"].max()

Timestamp('2022-01-17 00:00:00')

In [595]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: L2 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: L2 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)].groupby(["Case: L2 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Account Safety,444,0.790036,36.486486
Account Access,55,0.097865,34.545455
Account Risk,43,0.076512,20.930233
Other,6,0.010676,16.666667
Compliance Policy,5,0.008897,40.000000
Crypto Ops,5,0.008897,40.000000
Pro,2,0.003559,0.000000
API Support,1,0.001779,100.000000
ACH,1,0.001779,0.000000


#### 1st

In [596]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: L2 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: L2 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)].groupby(["Case: L2 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Account Safety,475,0.774878,30.315789
Account Access,62,0.101142,22.580645
Account Risk,53,0.086460,18.867925
Other,12,0.019576,25.000000
Compliance Policy,3,0.004894,0.000000
Pro,2,0.003263,50.000000
Wallet,2,0.003263,0.000000
Scaled Developer Support,1,0.001631,0.000000
ACH,1,0.001631,0.000000
SEPA,1,0.001631,0.000000


### L3 Workflow

#### 4th

In [597]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
User Disable Signin,326,0.580071,41.411043
Unauthorized Password Reset,31,0.055160,38.709677
Unauthorized Crypto Buy/Sell/Send,28,0.049822,10.714286
Self Recovery Applied,20,0.035587,40.000000
CX Email Compromised,13,0.023132,7.692308
Clawback Process,12,0.021352,16.666667
Phishing Scam Victim,9,0.016014,44.444444
Remote / Physical Device Takeovers,7,0.012456,14.285714
User Sent Funds to Scammer,7,0.012456,14.285714
Disable TOTP or Authy,7,0.012456,28.571429


#### 1st

In [598]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
User Disable Signin,328,0.535073,34.451220
Unauthorized Crypto Buy/Sell/Send,47,0.076672,4.255319
CX Email Compromised,22,0.035889,31.818182
WBL Hold Time,13,0.021207,7.692308
Unauthorized Password Reset,13,0.021207,53.846154
Clawback Process,13,0.021207,23.076923
Troubleshooting,13,0.021207,38.461538
Self Recovery Applied,10,0.016313,10.000000
User Sent Funds to Scammer,10,0.016313,10.000000
SIM Swap / Phone Port,10,0.016313,0.000000


### Macro

#### 4th

In [599]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Already Processing,149,0.264184,53.020134
Resolved - Self Recovery,108,0.191489,33.333333
General > Merged Case,51,0.090426,21.568627
AR Already Processing,20,0.035461,30.000000
Temporary - Trust - Self Recovery Completed,20,0.035461,10.000000
Resolved - Restoring,13,0.023050,46.153846
Self Recovery - Escalate to Queue,12,0.021277,50.000000
Password Reset,10,0.017730,20.000000
Resolved - Customer Replaced Device - TOTP Disable,8,0.014184,25.000000
T1 Pro General > Free Form Template #1 - General,8,0.014184,12.500000


#### 1st

In [600]:
pd.concat([DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Already Processing,146,0.237785,43.835616
Resolved - Self Recovery,83,0.135179,22.891566
General > Merged Case,38,0.061889,18.421053
AR Already Processing,23,0.037459,47.826087
Self Recovery - Escalate to Queue,20,0.032573,40.000000
Temporary - Trust - Self Recovery Completed,19,0.030945,15.789474
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,16,0.026059,31.250000
Trust - T1 - Secure Account,16,0.026059,25.000000
2FA - Resolved - Self Recovery,14,0.022801,35.714286
Silent Resolve,10,0.016287,20.000000


In [623]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Applied,20,0.363636,40.000000
Disable TOTP or Authy,7,0.127273,28.571429
Troubleshooting,6,0.109091,16.666667
Loses Access to 2FA - TOTP or Authy,5,0.090909,40.000000
FAQ: The Password Reset Process,5,0.090909,40.000000
Missing Device Confirmation Emails,5,0.090909,40.000000
Customer Loses Access to 2FA - SMS,3,0.054545,0.000000
IDV - Jumio Failed (Name / Country),2,0.036364,50.000000
Invalid 2FA Codes,1,0.018182,0.000000
No IDV Upload,1,0.018182,100.000000


In [624]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Troubleshooting,13,0.209677,38.461538
Self Recovery Applied,10,0.161290,10.000000
Customer Loses Access to 2FA - SMS,9,0.145161,22.222222
Loses Access to 2FA - TOTP or Authy,5,0.080645,0.000000
Disable TOTP or Authy,4,0.064516,50.000000
Missing Device Confirmation Emails,4,0.064516,75.000000
FAQ: The Password Reset Process,3,0.048387,0.000000
SMS Phone Change,2,0.032258,0.000000
Email Verification Failed,2,0.032258,0.000000
No IDV Upload,2,0.032258,0.000000


In [678]:
DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].to_excel("filter_this.xlsx",index=False)

In [679]:
DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].to_excel("filter_this2.xlsx",index=False)

In [698]:
DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].to_excel("Account-Safety.xlsx")

In [699]:
DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].to_excel("Account-Safety2.xlsx")

In [672]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
User Disable Signin,326,0.734234,41.411043
Unauthorized Password Reset,31,0.069820,38.709677
Unauthorized Crypto Buy/Sell/Send,28,0.063063,10.714286
CX Email Compromised,13,0.029279,7.692308
Phishing Scam Victim,9,0.020270,44.444444
Remote / Physical Device Takeovers,7,0.015766,14.285714
User Sent Funds to Scammer,7,0.015766,14.285714
Comprised 2FA Codes,6,0.013514,50.000000
SIM Swap / Phone Port,5,0.011261,20.000000
Phone Scam Victim,4,0.009009,0.000000


In [ ]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

In [673]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: L3 Workflow"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: L3 Workflow"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].groupby(["Case: L3 Workflow"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
User Disable Signin,328,0.690526,34.451220
Unauthorized Crypto Buy/Sell/Send,47,0.098947,4.255319
CX Email Compromised,22,0.046316,31.818182
Unauthorized Password Reset,13,0.027368,53.846154
User Sent Funds to Scammer,10,0.021053,10.000000
SIM Swap / Phone Port,10,0.021053,0.000000
Phishing Scam Victim,9,0.018947,22.222222
Remote / Physical Device Takeovers,9,0.018947,22.222222
Phone Port Account Takeovers,7,0.014737,14.285714
Comprised 2FA Codes,7,0.014737,85.714286


In [608]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Already Processing,8,0.145455,37.500000
Resolved - Restoring,4,0.072727,50.000000
Happy Place - Thank You,3,0.054545,100.000000
Temporary - Trust - Self Recovery Completed,3,0.054545,33.333333
AR Already Processing,3,0.054545,33.333333
General > Merged Case,3,0.054545,33.333333
Email - No Verification Email in SPAM Folder,2,0.036364,50.000000
Account Recovery - FM Troubleshooting,2,0.036364,0.000000
Resolved - Customer Replaced Device - TOTP Disable,2,0.036364,0.000000
2FA - Resolved - Self Recovery,2,0.036364,0.000000


In [609]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Access")].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,8,0.129032,37.500000
2FA - Phone Change (SMS),6,0.096774,16.666667
Email - No Verification Email in SPAM Folder,4,0.064516,75.000000
AR Already Processing,3,0.048387,0.000000
2FA - Resolved - Self Recovery,3,0.048387,0.000000
Self Recovery Already Processing,3,0.048387,33.333333
2FA - TOTP Disable,3,0.048387,33.333333
Self Recovery - Escalate to Queue,2,0.032258,50.000000
2FA - TOTP Troubleshooting,2,0.032258,0.000000
Silent Resolve,2,0.032258,0.000000


In [674]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_4th) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Already Processing,139,0.313770,54.676259
Resolved - Self Recovery,107,0.241535,32.710280
General > Merged Case,44,0.099323,20.454545
AR Already Processing,17,0.038375,29.411765
Temporary - Trust - Self Recovery Completed,16,0.036117,6.250000
Self Recovery - Escalate to Queue,11,0.024831,45.454545
Password Reset,9,0.020316,22.222222
Resolved - Restoring,7,0.015801,28.571429
T1 Pro General > Free Form Template #1 - General,6,0.013544,16.666667
Resolved - Customer Replaced Device - TOTP Disable,6,0.013544,33.333333


In [675]:
pd.concat([DF_SURVEY[(DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st)) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: Last Macro Applied"].value_counts().rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")]["Case: Last Macro Applied"].value_counts(normalize=True).rename("count"),
           DF_SURVEY[DF_SURVEY["Agent_Name_Cleaned"].isin(Survey_1st) & (DF_SURVEY["Case: L2 Workflow"] == "Account Safety")].groupby(["Case: Last Macro Applied"])["Resolution"].mean()], axis=1).head(10)

,count,count,Resolution
Self Recovery Already Processing,142,0.298947,44.366197
Resolved - Self Recovery,83,0.174737,22.891566
General > Merged Case,33,0.069474,21.212121
AR Already Processing,20,0.042105,55.000000
Temporary - Trust - Self Recovery Completed,19,0.040000,15.789474
Self Recovery - Escalate to Queue,18,0.037895,38.888889
Trust - T1 - Secure Account,15,0.031579,26.666667
2FA - Resolved - Self Recovery,10,0.021053,40.000000
1st Response - Hacked - Disabled Sign in,8,0.016842,12.500000
Balance removed - account closed,7,0.014737,57.142857


In [610]:
DF_SURVEY

,Case Number,Was your issue resolved?,Additional Feedback,Case: Submitted Category,Case: Category,Case: Subcategory,Case: Last Macro Applied,Survey Response: Created Date,Case: Survey Sent Date/Time,Case: Date/Time Opened,...,Case: Sub Team,Case: Team,Survey Response: Owner Name,Case: Case Owner Email,How can we do better?,What did we do well?,Agent_Name_Cleaned,Survey_Month,Resolution,Survey Week
0,9068437,No,I wish to be compensated the money lost during...,NaN,Payments - General,Fees,Crypto - Conversion Spread,2021-12-22,12/22/2021 9:15 AM,12/14/2021 3:37 PM,...,Crypto,Support - Payments,Johanna Bangga,johanna.bangga@coinbase.com,NaN,NaN,johanna bangga,December,0,2021-12-20
1,8951509,Yes,Stop sharing user data.,data_privacy,Compliance,I would like to make a personal data request,Privacy - Data Sale,2021-12-07,12/7/2021 7:45 PM,12/7/2021 3:19 PM,...,Compliance,Policy Ops,Christopher Catada,christopher.catada@coinbase.com,NaN,NaN,christopher catada,December,100,2021-12-06
2,9004792,No,still waiting for answers.,NaN,Payments - General,Fees,Crypto - Conversion Spread,2021-12-10,12/10/2021 1:30 PM,12/10/2021 11:26 AM,...,Payments,BPO Ops,Sedney Baricuatro,sedney.baricuatro@coinbase.com,NaN,NaN,sedney baricuatro,December,0,2021-12-06
3,9032711,No,"Hey,\n\nThanks but I am more interested in a s...",Account_Statement_Report,Payments - General,Product Inquiry,Pro > Managing Portfolios,2021-12-13,12/12/2021 5:00 PM,12/12/2021 3:13 PM,...,Compliance,Policy Ops,april rizo,richarddominic.monasterio@coinbase.com,NaN,NaN,april rizo,December,0,2021-12-13
4,8839670,No,Why comment you guys dont do SHIT?\n98K7MD6Q i...,Fees_Limits,Payments - General,Fees,Crypto - Conversion Spread,2021-12-01,12/1/2021 2:45 PM,12/1/2021 9:11 AM,...,Risk,Account Ops,danica sapitula,danica.sapitula@coinbase.com,NaN,NaN,danica sapitula,December,0,2021-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45039,8128936,No,Why won’t Coinbase recognize that ethsany.org ...,NaN,Fraud,NaN,Fraud > Account creation questions,2021-11-24,11/24/2021 5:30 PM,10/28/2021 7:55 AM,...,Safety,Account Ops,Bianca Acuna,bianca.acuna@coinbase.com,NaN,NaN,bianca acuna,November,0,2021-11-22
45040,8742831,No,"Screenshots added, fees just raised when I red...",Send_Receive_Crypto,Send/Receive Crypto,Outgoing transactions from my Coinbase Wallet,NaN,2021-11-27,11/26/2021 2:15 PM,11/25/2021 3:29 PM,...,Crypto,Support - Payments,Rhea Boter,till.burkhardt@coinbase.com,NaN,NaN,rhea boter,November,0,2021-11-22
45041,8598886,No,NaN,Send_Receive_Crypto,Other,Other,NaN,2021-11-19,11/18/2021 1:45 PM,11/18/2021 9:10 AM,...,Other,Support - Wallet,Clarissa Castromero,till.burkhardt@coinbase.com,NaN,NaN,clarissa castromero,November,0,2021-11-15
45042,8430401,No,Not been resolved yet.,Linking_a_Payment_Method,Fraud,Banned User,NaN,2021-11-09,11/9/2021 4:00 PM,11/9/2021 12:44 PM,...,Payments,BPO Ops,jonas santos,jonas.santos@coinbase.com,NaN,NaN,jonas santos,November,0,2021-11-08


In [618]:
DF_SURVEY["Case: L1 Workflow"].value_counts()

Identity Ops         23518
Policy Ops            6307
Ecosystem Ops         5339
Payment Ops           4850
Retail Ops            4510
Institutional Ops      198
Trading                122
Banking Ops            121
General                 67
Account Ops              7
Community Ops            1
Name: Case: L1 Workflow, dtype: int64

In [622]:
DF_SURVEY[DF_SURVEY["Case: L1 Workflow"] == "Retail Ops"]["Case: L3 Workflow"].value_counts()

Eligibility Requirements                           1377
Referrals, Incentives, Sweepstakes                  967
Wallet Quick Wins                                   747
App Bugs                                            238
Transferring between Wallet and Retail              176
Wallet Account Management                           143
Rewards                                             129
Consumer Transfer                                   113
Recurring Transactions                              111
Coinbase Card                                        98
Account Access                                       85
Dapp Browser Troubleshooting                         51
Vault Withdrawals                                    49
Buying Crypto Using Linked coinbase.com Account      37
Multiple Issues                                      32
Borrow                                               31
Press Workflow                                       28
Coinbase One Questions                          

In [ ]:
SANITY_CHECK = \
NEW_QUERY.merge(DF_SCHED_DEC, on="Agent_Name_Cleaned")

In [641]:
COACHING_DATA = \
AGENT_PERFORMANCE.reset_index().merge(DF_COACHING, on="Agent_Name_Cleaned")

COACHING_DATA = COACHING_DATA[~(COACHING_DATA["Completion%"].isnull())].reset_index(drop=True)

In [660]:
TO_CHECK = COACHING_DATA["Completion%"]

CONDITIONS = [(TO_CHECK < 0.3),
              (TO_CHECK >= 0.3) & (TO_CHECK < 0.5),
              (TO_CHECK >= 0.5) & (TO_CHECK < 0.8),
              (TO_CHECK >= 0.8)
             ]

CHOICES = ["< 30%", "< 50%", "< 80%", "< 100%"]

COACHING_DATA["Coaching_Bin"] = np.select(CONDITIONS, CHOICES,np.nan)

In [671]:
pd.concat([COACHING_DATA.groupby(["Coaching_Bin"])[["Score","Resolution_Survey", "Completion%"]].mean(),
           COACHING_DATA.groupby(["Coaching_Bin"])["Resolution_Survey"].count().rename("Count"),
           COACHING_DATA.groupby(["Coaching_Bin"])["Agent_Name_Cleaned"].nunique().rename("Count")
          ],axis=1)

,Score,Resolution_Survey,Completion%,Count,Count
Coaching_Bin,,,,,
< 100%,95.923263,32.180736,0.943571,28,28
< 30%,93.988095,21.428571,0.092857,7,7
< 50%,82.800000,29.333333,0.364000,5,5
< 80%,90.795455,20.600649,0.619167,12,12


In [668]:
COACHING_DATA.groupby(["Coaching_Bin"])["Resolution_Survey"].count().rename("Count")

Coaching_Bin
< 100%    28
< 30%      7
< 50%      5
< 80%     12
Name: Resolution_Survey, dtype: int64

In [687]:
COACHING_DATA

,Agent_Name_Cleaned,Greeting,Active Listening & Acknowledgement,Customer Engagement,Probing & Validation,Resolution,Closing & Recap,Call Pace & Hold Procedure,Case & Account Comments,"Case Labels, Workflow Picklist, Case Status",With CE?,Did the Analyst able to complete phone authentication and verify caller?,Did the Analyst avoid disclosing MNPI or private information?,Score,Resolution_Survey,Survey_Quartile,QAMonitoring_Quartile,Completion%,Compliance%,Coaching_Bin
0,abegail puerta,100.0,90.909091,90.909091,100.000000,100.000000,90.909091,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000,96.363636,45.454545,3rd,3rd,0.67,0.67,< 80%
1,agnes mae morales,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000,100.000000,0.000000,1st,3rd,1.00,1.00,< 100%
2,anecito veldosola,100.0,100.000000,100.000000,83.333333,100.000000,100.000000,100.000000,100.000000,100.000000,16.666667,83.333333,100.000000,97.500000,16.666667,2nd,3rd,0.33,0.28,< 50%
3,carl yumol,100.0,100.000000,100.000000,50.000000,50.000000,50.000000,50.000000,100.000000,100.000000,50.000000,50.000000,100.000000,72.500000,0.000000,1st,1st,0.68,0.63,< 80%
4,chad cledera,100.0,100.000000,90.909091,72.727273,81.818182,90.909091,100.000000,90.909091,90.909091,9.090909,90.909091,90.909091,88.181818,18.181818,2nd,1st,0.57,0.57,< 80%
5,charles angeles,100.0,100.000000,91.666667,91.666667,100.000000,91.666667,91.666667,100.000000,91.666667,8.333333,91.666667,100.000000,95.833333,8.333333,2nd,3rd,0.84,0.95,< 100%
6,charles irlandez,100.0,100.000000,50.000000,50.000000,50.000000,100.000000,100.000000,75.000000,100.000000,0.000000,100.000000,100.000000,71.250000,0.000000,1st,1st,0.29,0.24,< 30%
7,christian galinea,100.0,100.000000,100.000000,50.000000,100.000000,100.000000,100.000000,100.000000,50.000000,0.000000,100.000000,100.000000,90.000000,100.000000,4th,2nd,0.00,0.00,< 30%
8,dennis talacay,100.0,83.333333,83.333333,66.666667,66.666667,83.333333,100.000000,100.000000,100.000000,16.666667,100.000000,100.000000,80.000000,16.666667,2nd,1st,0.58,0.74,< 80%
9,edmer rabaya,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000,100.000000,33.333333,3rd,3rd,1.00,0.33,< 100%


In [680]:
AGENT_PERFORMANCE

,Greeting,Active Listening & Acknowledgement,Customer Engagement,Probing & Validation,Resolution,Closing & Recap,Call Pace & Hold Procedure,Case & Account Comments,"Case Labels, Workflow Picklist, Case Status",With CE?,Did the Analyst able to complete phone authentication and verify caller?,Did the Analyst avoid disclosing MNPI or private information?,Score,Resolution_Survey,Survey_Quartile,QAMonitoring_Quartile
Agent_Name_Cleaned,,,,,,,,,,,,,,,,
abegail puerta,100.0,90.909091,90.909091,100.0,100.000000,90.909091,100.0,100.0,100.0,0.0,100.0,100.0,96.363636,45.454545,3rd,3rd
abelardo gomezjr,100.0,100.000000,100.000000,100.0,100.000000,100.000000,0.0,100.0,100.0,0.0,100.0,100.0,95.000000,0.000000,1st,2nd
agnes mae morales,100.0,100.000000,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,0.0,100.0,100.0,100.000000,0.000000,1st,3rd
agustin bagguatan,100.0,100.000000,100.000000,100.0,66.666667,66.666667,100.0,100.0,100.0,0.0,100.0,100.0,88.333333,0.000000,1st,1st
amelia zuniga,100.0,100.000000,100.000000,100.0,50.000000,100.000000,100.0,100.0,100.0,50.0,100.0,100.0,87.500000,0.000000,1st,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
veronica badillo,100.0,100.000000,100.000000,0.0,0.000000,100.000000,100.0,0.0,0.0,100.0,100.0,100.0,50.000000,100.000000,4th,1st
victoria badillo,100.0,0.000000,100.000000,0.0,100.000000,100.000000,100.0,100.0,100.0,0.0,100.0,100.0,70.000000,0.000000,1st,1st
virginia leonin,100.0,100.000000,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,0.0,100.0,100.0,100.000000,25.000000,2nd,3rd


In [685]:
TENURE_DATA = \
AGENT_PERFORMANCE.reset_index().merge(DF_TENURE[~(DF_TENURE["Tenure in Pillar"].isnull())], on="Agent_Name_Cleaned")

#COACHING_DATA = COACHING_DATA[~(COACHING_DATA["Completion%"].isnull())].reset_index(drop=True)

In [696]:
AGENT_PERFORMANCE.shape

(101, 16)

In [694]:
TENURE_DATA

,Agent_Name_Cleaned,Greeting,Active Listening & Acknowledgement,Customer Engagement,Probing & Validation,Resolution,Closing & Recap,Call Pace & Hold Procedure,Case & Account Comments,"Case Labels, Workflow Picklist, Case Status",With CE?,Did the Analyst able to complete phone authentication and verify caller?,Did the Analyst avoid disclosing MNPI or private information?,Score,Resolution_Survey,Survey_Quartile,QAMonitoring_Quartile,Tenure in Pillar,Latest Training Wave
0,abegail puerta,100.0,90.909091,90.909091,100.000000,100.000000,90.909091,100.000000,100.000000,100.000000,0.000000,100.000000,100.0,96.363636,45.454545,3rd,3rd,>90d,Voice ATO 1 - T2 Access A
1,agustin bagguatan,100.0,100.000000,100.000000,100.000000,66.666667,66.666667,100.000000,100.000000,100.000000,0.000000,100.000000,100.0,88.333333,0.000000,1st,1st,>90d,Voice ATO 6
2,antoinette sotelo,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,77.777778,100.000000,88.888889,11.111111,100.000000,100.0,98.333333,44.444444,3rd,3rd,>90d,Voice ATO 1
3,arvin roallos,100.0,100.000000,66.666667,100.000000,83.333333,100.000000,66.666667,100.000000,100.000000,0.000000,100.000000,100.0,89.166667,16.666667,2nd,2nd,>90d,Voice ATO 4 - T2 Access B
4,bernard eugenio,100.0,100.000000,87.500000,100.000000,87.500000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.0,95.000000,25.000000,2nd,2nd,>90d,Voice ATO 1
5,bienca navarro,100.0,100.000000,100.000000,83.333333,83.333333,83.333333,83.333333,100.000000,100.000000,0.000000,100.000000,100.0,90.833333,50.000000,4th,2nd,>90d,Voice ATO 5
6,cheryle delacruz,100.0,90.909091,81.818182,90.909091,90.909091,100.000000,90.909091,90.909091,90.909091,0.000000,100.000000,100.0,90.909091,36.363636,3rd,2nd,>90d,Voice ATO 6
7,christian flores,100.0,100.000000,83.333333,100.000000,100.000000,83.333333,100.000000,100.000000,100.000000,0.000000,100.000000,100.0,95.833333,66.666667,4th,3rd,>90d,Voice ATO 4 - T2 Access A
8,edmer rabaya,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.0,100.000000,33.333333,3rd,3rd,>90d,Voice ATO 3 - T2 Access F
9,eleonor baniqued,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,85.714286,0.000000,100.000000,100.0,99.285714,57.142857,4th,3rd,>90d,Voice ATO 2


In [695]:
TENURE_DATA["Latest Training Wave"].value_counts()

Voice ATO 1                  6
Voice ATO 5                  4
Voice ATO 4 - T2 Access A    3
Voice ATO 6                  2
Voice ATO 2                  2
Voice ATO 2 - T2 Access D    2
Voice ATO 1 - T2 Access A    1
Voice ATO 4 - T2 Access B    1
Voice ATO 3 - T2 Access F    1
Voice ATO 4 - T2 Access F    1
Voice ATO 3 - T2 Access A    1
Voice ATO 4 - T2 Access C    1
Voice ATO 3 - T2 Access B    1
Name: Latest Training Wave, dtype: int64

In [693]:
TENURE_DATA.shape

(26, 19)

## Sanity Check

In [627]:
DF_SCHED_DEC = pd.read_excel("../Data/Other Details/SpaceJam Dec 2021.xlsx", sheet_name="Schedule File", skiprows=1)

In [676]:
DF_SCHED_DEC

,Site,Pillar & Prio,Current Support,Assignment,Tenure,Direct Supervisor,EID,Schedule,RD,2021-11-28 00:00:00,...,2021-12-22 00:00:00,2021-12-23 00:00:00,2021-12-24 00:00:00,2021-12-25 00:00:00,2021-12-26 00:00:00,2021-12-27 00:00:00,2021-12-28 00:00:00,2021-12-29 00:00:00,2021-12-30 00:00:00,2021-12-31 00:00:00
0,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,amiel.t.manuel,03:00 PM - 00:00 AM,Sat - Sun,OFF,...,03:00 PM - 00:00 AM,VL,VL,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM
1,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,lalaine.c.timple,03:00 PM - 00:00 AM,Tue - Wed,03:00 PM - 00:00 AM,...,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM
2,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,dona.c.tomboc,03:00 PM - 00:00 AM,Thu - Fri,03:00 PM - 00:00 AM,...,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF
3,Manila,T2 Sanction,Email T1,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,bernadette.s.cornel,06:00 PM - 03:00 AM,Sat - Sun,OFF,...,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,VL,OFF,OFF,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM
4,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,avegail.l.recaido,03:00 PM - 00:00 AM,Sat - Sun,OFF,...,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,VL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,Manila,ATO,Voice,Voice ATO 9,NaN,keena.kryss.uding,ma.geraldine.batang,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867,Manila,ATO,Voice,Voice ATO 9,NaN,keena.kryss.uding,mark.pineda,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,Manila,ATO,Voice,Voice ATO 9,NaN,keena.kryss.uding,sarah.detera,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,Manila,ATO,Voice,Voice ATO 9,NaN,keena.kryss.uding,vincent.luyo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [628]:
DF_SCHED_DEC["Tenure"].value_counts()

>90d          650
31-60d         45
>90             2
ATO Wave 8      1
Name: Tenure, dtype: int64

In [448]:
DF_SCHED_DEC.head()

,Site,Pillar & Prio,Current Support,Assignment,Tenure,Direct Supervisor,EID,Schedule,RD,2021-11-28 00:00:00,...,2021-12-22 00:00:00,2021-12-23 00:00:00,2021-12-24 00:00:00,2021-12-25 00:00:00,2021-12-26 00:00:00,2021-12-27 00:00:00,2021-12-28 00:00:00,2021-12-29 00:00:00,2021-12-30 00:00:00,2021-12-31 00:00:00
0,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,amiel.t.manuel,03:00 PM - 00:00 AM,Sat - Sun,OFF,...,03:00 PM - 00:00 AM,VL,VL,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM
1,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,lalaine.c.timple,03:00 PM - 00:00 AM,Tue - Wed,03:00 PM - 00:00 AM,...,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM
2,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,dona.c.tomboc,03:00 PM - 00:00 AM,Thu - Fri,03:00 PM - 00:00 AM,...,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF
3,Manila,T2 Sanction,Email T1,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,bernadette.s.cornel,06:00 PM - 03:00 AM,Sat - Sun,OFF,...,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,VL,OFF,OFF,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM,06:00 PM - 03:00 AM
4,Manila,Account Recovery Manual Review,CBV,Sanctions Screening Wave 2,>90d,glenn.c.hermoso,avegail.l.recaido,03:00 PM - 00:00 AM,Sat - Sun,OFF,...,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,OFF,OFF,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,03:00 PM - 00:00 AM,VL


In [449]:
DF_SCHED_DEC["Agent_Name_Cleaned"] = DF_SCHED_DEC["EID"].str.split(".").apply(lambda x: x[0] + " " + x[-1]).str.lower()

In [450]:
col_list_dec = DF_SCHED_DEC.columns[12:].to_list()

In [629]:
DF_COACHING

,Agent_Name_Cleaned,Completion%,Compliance%
0,ma. elvie nalayog,NaN,NaN
1,alavinia abdah,NaN,NaN
2,nathalia lachica,1.00,0.50
3,heherson juego,1.00,0.00
4,joy abanil,1.00,2.00
...,...,...,...
89,jackielyn mangahas,0.31,0.46
90,anecito veldosola,0.33,0.28
91,john lidnel siochi,0.25,0.31
92,charles irlandez,0.29,0.24


In [451]:
col_list_dec = DF_SCHED_DEC.columns[12:].to_list()
DF_SCHED_DEC[col_list_dec] = DF_SCHED_DEC[col_list_dec].replace("OFF", np.nan)
DF_SCHED_DEC[col_list_dec] = DF_SCHED_DEC[col_list_dec].replace("ML", np.nan)
DF_SCHED_DEC[col_list_dec] = DF_SCHED_DEC[col_list_dec].replace("VL", np.nan)
DF_SCHED_DEC[col_list_dec] = DF_SCHED_DEC[col_list_dec].replace("ML~", np.nan)

In [452]:
for col in col_list_dec:
    DF_SCHED_DEC[col] = DF_SCHED_DEC[col].str.replace("|", "-", regex=True)
    DF_SCHED_DEC[col] = DF_SCHED_DEC[col].str.replace("^", "-", regex=True)
    DF_SCHED_DEC[col] = DF_SCHED_DEC[col].str.replace(":", "-", regex=True)
    DF_SCHED_DEC[col] = DF_SCHED_DEC[col].str.replace("~", "-", regex=True)

In [453]:
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC[col_list_dec[0]].fillna(DF_SCHED_DEC[col_list_dec[1]])
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].fillna(DF_SCHED_DEC[col_list_dec[2]])
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].fillna(DF_SCHED_DEC[col_list_dec[3]])
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].fillna(DF_SCHED_DEC[col_list_dec[4]])
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].fillna(DF_SCHED_DEC[col_list_dec[5]])
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].fillna(DF_SCHED_DEC[col_list_dec[6]])

In [454]:
DF_SCHED_DEC = DF_SCHED_DEC[~(DF_SCHED_DEC["Schedule_Ref"].isnull())].reset_index(drop=True)

In [456]:
DF_SCHED_DEC["Schedule_Ref"] = DF_SCHED_DEC["Schedule_Ref"].str.split("-").apply(lambda x: x[0] + ":" + x[1]).apply(pd.to_datetime)

In [457]:
TO_CHECK = DF_SCHED_DEC["Schedule_Ref"].dt.hour

CONDITIONS = [(TO_CHECK >= 6) & (TO_CHECK < 15),
              (TO_CHECK >= 15) & (TO_CHECK < 23)
             ]

CHOICES = ["Morning", "Mid"]

DF_SCHED_DEC["Shift Schedule Workforce"] = np.select(CONDITIONS, CHOICES, "Night")

In [459]:
DF_SCHED_DEC = DF_SCHED_DEC[["Agent_Name_Cleaned", "Direct Supervisor", "Shift Schedule Workforce"]]

In [460]:
DF_SCHED_DEC.head()

,Agent_Name_Cleaned,Direct Supervisor,Shift Schedule Workforce
0,amiel manuel,glenn.c.hermoso,Mid
1,lalaine timple,glenn.c.hermoso,Mid
2,dona tomboc,glenn.c.hermoso,Mid
3,bernadette cornel,glenn.c.hermoso,Mid
4,avegail recaido,glenn.c.hermoso,Mid


In [462]:
SANITY_CHECK = \
NEW_QUERY.merge(DF_SCHED_DEC, on="Agent_Name_Cleaned")

In [467]:
SANITY_CHECK.shape

(1625, 41)

In [468]:
NEW_QUERY.shape

(1816, 39)

In [470]:
SANITY_CHECK = SANITY_CHECK[SANITY_CHECK["Survey_Month"] == "December"].reset_index(drop=True)

In [475]:
pd.concat([SANITY_CHECK.groupby(["Shift Schedule Workforce"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK.groupby(["Shift Schedule Workforce"])["Resolution"].count().rename("Count"),
           SANITY_CHECK.groupby(["Shift Schedule Workforce"])["Agent_Name_Cleaned"].nunique().rename("Agent on Shift")
          ], axis=1
         )

,Resolution %,Count,Agent on Shift
Shift Schedule Workforce,,,
Mid,29.928741,421,32
Morning,39.047619,210,20
Night,38.116592,223,19


In [476]:
SANITY_CHECK

,Case Number,Was your issue resolved?,Additional Feedback,Case: Submitted Category,Case: Category,Case: Subcategory,Case: Last Macro Applied,Survey Response: Created Date,Case: Survey Sent Date/Time,Case: Date/Time Opened,...,Case: Case Owner Email,How can we do better?,What did we do well?,Agent_Name_Cleaned,Survey_Month,Resolution,Survey Week,Shift Schedule,Direct Supervisor,Shift Schedule Workforce
0,9199751,No,After the hack money was stolen from me and yo...,NaN,I need help with my email or password,NaN,Duplicate Case - Repeat Request,2021-12-24,12/24/2021 11:30 AM,12/22/2021 3:26 PM,...,dennis.talacay@coinbase.com,NaN,NaN,dennis talacay,December,0,2021-12-20,Morning,jasmine.t.alcantara,Night
1,9002072,Yes,NaN,NaN,I need help with my email or password,NaN,Self Recovery Already Processing,2021-12-10,12/10/2021 10:15 AM,12/10/2021 8:52 AM,...,dennis.talacay@coinbase.com,NaN,NaN,dennis talacay,December,100,2021-12-06,Mid,jasmine.t.alcantara,Night
2,8967791,Yes,NaN,NaN,I need help with my email or password,NaN,Self Recovery Already Processing,2021-12-09,12/8/2021 1:00 PM,12/8/2021 11:34 AM,...,dennis.talacay@coinbase.com,NaN,NaN,dennis talacay,December,100,2021-12-06,Mid,jasmine.t.alcantara,Night
3,8825708,No,NaN,NaN,I need help with my email or password,NaN,Self Recovery Already Processing,2021-12-01,11/30/2021 4:30 PM,11/30/2021 3:12 PM,...,dennis.talacay@coinbase.com,NaN,NaN,dennis talacay,December,0,2021-11-29,Mid,jasmine.t.alcantara,Night
4,8943628,No,$964.29 was taken and you only returned $835.6...,NaN,Account Compromise,NaN,General questions about Escheatment,2021-12-08,12/8/2021 11:00 AM,12/7/2021 8:14 AM,...,dennis.talacay@coinbase.com,NaN,NaN,dennis talacay,December,0,2021-12-06,Mid,jasmine.t.alcantara,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,8842927,No,NaN,NaN,Account,NaN,Resolved - Self Recovery,2021-12-02,12/1/2021 1:00 PM,12/1/2021 11:51 AM,...,jamila.pongcol@coinbase.com,NaN,NaN,jamila pongcol,December,0,2021-11-29,Mid,ana.m.v.bongcaron,Night
850,8911879,No,Still locked out waiting for verification,NaN,Account Compromise,Lost/Stolen device,Resolved - Customer Replaced Device - TOTP Dis...,2021-12-05,12/5/2021 1:00 PM,12/5/2021 11:30 AM,...,jamila.pongcol@coinbase.com,NaN,NaN,jamila pongcol,December,0,2021-11-29,Mid,ana.m.v.bongcaron,Night
851,9035531,Yes,Thank you!,NaN,Account,NaN,Resolved - Self Recovery,2021-12-12,12/12/2021 11:15 PM,12/12/2021 9:50 PM,...,sandra.condor@coinbase.com,NaN,NaN,sandra condor,December,100,2021-12-06,Mid,alavinia.u.abdah,Night
852,8871523,No,NaN,NaN,Account,NaN,Resolved - Self Recovery,2021-12-04,12/3/2021 12:00 AM,12/2/2021 10:32 PM,...,maricel.o.osera_external.acn@coinbase.com,NaN,NaN,renalie quijano,December,0,2021-11-29,Mid,eddelyn.v.t.bendana,Morning


In [477]:
pd.concat([SANITY_CHECK.groupby(["Direct Supervisor"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK.groupby(["Direct Supervisor"])["Resolution"].count().rename("Count"),
           SANITY_CHECK.groupby(["Direct Supervisor"])["Agent_Name_Cleaned"].nunique().rename("Agent on Shift")
          ], axis=1
         )

,Resolution %,Count,Agent on Shift
Direct Supervisor,,,
agnes.mae.l.morales,24.691358,81,8
alavinia.u.abdah,42.207792,154,12
ana.m.v.bongcaron,33.333333,84,9
eddelyn.v.t.bendana,33.050847,118,12
jasmine.t.alcantara,29.220779,154,12
jobell.s.alvarez,37.078652,178,12
reenberly.s.bombasi,35.294118,85,6


In [481]:
SANITY_CHECK.groupby(["Direct Supervisor", "Shift Schedule Workforce"])["Agent_Name_Cleaned"].nunique().unstack().fillna(0).astype(int)

Shift Schedule Workforce,Mid,Morning,Night
Direct Supervisor,,,
agnes.mae.l.morales,4,4,0
alavinia.u.abdah,1,5,6
ana.m.v.bongcaron,6,0,3
eddelyn.v.t.bendana,2,9,1
jasmine.t.alcantara,3,0,9
jobell.s.alvarez,12,0,0
reenberly.s.bombasi,4,2,0


In [494]:
NIGHT_SHIFTER = \
pd.concat([SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Night"].groupby(["Case: L3 Workflow"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Night"].groupby(["Case: L3 Workflow"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

MID_SHIFTER = \
pd.concat([SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Mid"].groupby(["Case: L3 Workflow"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Mid"].groupby(["Case: L3 Workflow"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

MORNING_SHIFTER = \
pd.concat([SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Morning"].groupby(["Case: L3 Workflow"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[SANITY_CHECK["Shift Schedule Workforce"] == "Morning"].groupby(["Case: L3 Workflow"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

In [495]:
MID_SHIFTER

,Resolution %,Count
Case: L3 Workflow,,
User Disable Signin,38.647343,207
Unauthorized Crypto Buy/Sell/Send,6.451613,31
Unauthorized Password Reset,41.176471,17
Self Recovery Applied,31.250000,16
CX Email Compromised,30.769231,13
Clawback Process,15.384615,13
Phishing Scam Victim,36.363636,11
TextMatch,0.000000,8
WBL Hold Time,14.285714,7


In [496]:
MORNING_SHIFTER

,Resolution %,Count
Case: L3 Workflow,,
User Disable Signin,46.052632,152
Unauthorized Crypto Buy/Sell/Send,10.000000,10
Unauthorized Password Reset,33.333333,9
Self Recovery Applied,20.000000,5
Multiple Issues,0.000000,3
CX Email Compromised,0.000000,3
Session Compromise,50.000000,2
Clawback Process,0.000000,2
Credential Dump Monitoring Feature,50.000000,2


In [499]:
SANITY_CHECK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 41 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Case Number                           854 non-null    int64         
 1   Was your issue resolved?              854 non-null    object        
 2   Additional Feedback                   352 non-null    object        
 3   Case: Submitted Category              180 non-null    object        
 4   Case: Category                        794 non-null    object        
 5   Case: Subcategory                     209 non-null    object        
 6   Case: Last Macro Applied              848 non-null    object        
 7   Survey Response: Created Date         854 non-null    datetime64[ns]
 8   Case: Survey Sent Date/Time           854 non-null    object        
 9   Case: Date/Time Opened                854 non-null    object        
 10  Ca

In [505]:
NIGHT_SHIFTER_MACRO = \
pd.concat([SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Night") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin"].groupby(["Case: Last Macro Applied"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Night") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin"].groupby(["Case: Last Macro Applied"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

MID_SHIFTER_MACRO = \
pd.concat([SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Mid") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin	"].groupby(["Case: Last Macro Applied"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Mid") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin	"].groupby(["Case: Last Macro Applied"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

MORNING_SHIFTER_MACRO = \
pd.concat([SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Morning") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin	"].groupby(["Case: Last Macro Applied"])["Resolution"].mean().rename("Resolution %"),
           SANITY_CHECK[(SANITY_CHECK["Shift Schedule Workforce"] == "Morning") & SANITY_CHECK["Case: L3 Workflow"] == "User Disable Signin	"].groupby(["Case: Last Macro Applied"])["Resolution"].count().rename("Count")
          ],axis=1).sort_values(by="Count", ascending=False)

In [507]:
SANITY_CHECK.to_excel("Sanity_Check.xlsx",index=False)